In [2]:
import pandas as pd



In [3]:

invest_iter = pd.read_csv('crunchbase-investments.csv', chunksize = 5000, 
                          encoding = 'Latin-1')

missing_value = []
memory_usage = []
row_count = 0
for chunk in invest_iter:
    
    # Get missing values
    missing_value.append(chunk.isnull().sum())
    
    # Get memory usage
    memory_usage.append(chunk.memory_usage(deep=True))
    
    # Get row_count
    row_count += len(chunk)
    
# Combine and calculate missing values for all chunks
missing_values_combined = pd.concat(missing_value)
missing_values_total = missing_values_combined.groupby(level=0).sum()

# Combine and calculate total memory by column
memory_usage_combined = pd.concat(memory_usage)
memory_usage_total = memory_usage_combined.sum()/(1024*1024)

print("Missing values per column:")
print(missing_values_total)
print()
print("Total Rows =", row_count)
print()
print("Total Memory(MB) =", memory_usage_total)


Missing values per column:
company_category_code       643
company_city                533
company_country_code          1
company_name                  1
company_permalink             1
company_region                1
company_state_code          492
funded_at                     3
funded_month                  3
funded_quarter                3
funded_year                   3
funding_round_type            3
investor_category_code    50427
investor_city             12480
investor_country_code     12001
investor_name                 2
investor_permalink            2
investor_region               2
investor_state_code       16809
raised_amount_usd          3599
dtype: int64

Total Rows = 52870

Total Memory(MB) = 56.988484382629395


<b> Sample data to Determine which columns can be dropped<b>

In [4]:
invest = pd.read_csv('crunchbase-investments.csv', nrows = 5, 
                          encoding = 'Latin-1')

print(invest)

     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA                NaN  United States - Other   
3                  USA                 OH               Columbus   
4                  USA                 OH              Cleveland   

    company_city          investor_permalink      investor_name  \
0  San Francisco  /company/1-800-flowers-com  1-800-FLOWERS.COM   
1  Mountain View        /company/10xelerator        10Xelerator   
2            NaN       

NOTES:
- Columns that can be droppped: 
    - company_permalink: No extra info
    - company_country_code: Only two values
    - investor_permalink: No extra info
    - investor_category_code: too many missing values
    - investor_category_code; Maybe?

In [5]:
invest = pd.read_csv('crunchbase-investments.csv', nrows = 10, 
                          encoding = 'Latin-1')

print("Get Column data types")
print(invest.info())

Get Column data types
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
company_permalink         10 non-null object
company_name              10 non-null object
company_category_code     10 non-null object
company_country_code      10 non-null object
company_state_code        9 non-null object
company_region            10 non-null object
company_city              9 non-null object
investor_permalink        10 non-null object
investor_name             10 non-null object
investor_category_code    7 non-null object
investor_country_code     8 non-null object
investor_state_code       8 non-null object
investor_region           10 non-null object
investor_city             8 non-null object
funding_round_type        10 non-null object
funded_at                 10 non-null object
funded_month              10 non-null object
funded_quarter            10 non-null object
funded_year               10 non-null int64
raised_amount_usd         10

<b>Get column unique value counts to determine if any column can be converted to category data type <b>

In [6]:
usecols = ['company_name', 'company_category_code',
       'company_state_code', 'company_region',
       'company_city', 'investor_name',
       'investor_country_code',
       'investor_region', 'investor_city',
       'funding_round_type', 'funded_at', 'funded_month', 'funded_quarter',
       'funded_year', 'raised_amount_usd']
invest = pd.read_csv('crunchbase-investments.csv', chunksize = 5000, 
                          encoding = 'Latin-1', usecols = usecols )

column_unique_value = {}
for chunk in invest:
    for c in chunk.select_dtypes(include=['object']):
        
        unique_value = chunk[c].value_counts()

        if c in column_unique_value:
            column_unique_value[c].append(unique_value)
        else:
            column_unique_value[c] = [unique_value]

unique_value_final = {}
for c in column_unique_value.keys():
    combined_vc = pd.concat(column_unique_value[c])
    unique_value_final[c] = combined_vc.groupby(level = 0).sum()
    
for c in unique_value_final:
    
    if len(unique_value_final[c])/ unique_value_final[c].sum() < 0.5:
        print('Column =', c)
        print('Total unique values =', len(unique_value_final[c]))
        print('Total rows =', unique_value_final[c].sum() )
        print('Unique values % =', 
              (len(unique_value_final[c])/unique_value_final[c].sum()) * 100)
        print()


Column = company_state_code
Total unique values = 50
Total rows = 52378
Unique values % = 0.09545992592309749

Column = company_region
Total unique values = 546
Total rows = 52869
Unique values % = 1.0327413039777564

Column = investor_region
Total unique values = 585
Total rows = 52868
Unique values % = 1.106529469622456

Column = company_city
Total unique values = 1229
Total rows = 52337
Unique values % = 2.348243116724306

Column = funded_month
Total unique values = 192
Total rows = 52867
Unique values % = 0.3631755159173019

Column = company_name
Total unique values = 11573
Total rows = 52869
Unique values % = 21.889954415631088

Column = funded_at
Total unique values = 2808
Total rows = 52867
Unique values % = 5.31144192029054

Column = funded_quarter
Total unique values = 72
Total rows = 52867
Unique values % = 0.1361908184689882

Column = investor_city
Total unique values = 990
Total rows = 40390
Unique values % = 2.4511017578608567

Column = investor_country_code
Total unique v

In [7]:
import sqlite3

conn = sqlite3.connect('invest.db')
conn.execute('DROP TABLE invest')

usecols = ['company_name', 'company_category_code',
       'company_country_code', 'company_state_code', 'company_region',
       'company_city', 'investor_name',
       'investor_country_code',
       'investor_region', 'investor_city',
       'funding_round_type', 'funded_at', 'funded_month', 'funded_quarter',
       'funded_year', 'raised_amount_usd']
invest = pd.read_csv('crunchbase-investments.csv', chunksize = 5000, 
                          encoding = 'Latin-1', usecols = usecols )


mem_total = 0

for chunk in invest:
    
    # Try a better integer subtype
    chunk['funded_year'] = pd.to_numeric(chunk['funded_year'],
                                         downcast='integer') 
    chunk['raised_amount_usd'] = pd.to_numeric(chunk['raised_amount_usd'],
                                               downcast='integer') 
    
    
    # Remove extra data from funded_quarter (YYYY-)
    chunk['funded_quarter'] = chunk['funded_quarter'].str.slice(start = 5)
    
    # Cast columns to category data type
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('category')
    chunk['company_state_code'] = chunk['company_state_code'].astype('category')
    chunk['investor_city'] = chunk['investor_city'].astype('category')
    chunk['investor_country_code'] = chunk['investor_country_code'].astype('category')
    chunk['funded_month'] = chunk['funded_month'].astype('category')
    chunk['funding_round_type'] = chunk['funding_round_type'].astype('category')
    chunk['company_category_code'] = chunk['company_category_code'].astype('category')
    
    # Save each chunk to database
    dtypes = {'raised_amount_usd': 'integer'}
    chunk.to_sql('invest', conn, if_exists='append', index = False, dtype = dtypes )
    
    # Calculate chunk memory usage
    mem_total += chunk.memory_usage(deep = True).sum()

print("Total Memory =",mem_total/(1024*1024))






    
    






Total Memory = 24.866358757019043


<b>Check database size in disk:<b>

In [8]:
!wc invest.db

   5418  232543 6991872 invest.db


<b>Verify column data type in the database:<b>

In [9]:
# Check Table info                 
conn = sqlite3.connect('invest.db')
results_df = pd.read_sql('PRAGMA table_info(invest)', conn)

print(results_df)


    cid                   name     type  notnull dflt_value  pk
0     0           company_name     TEXT        0       None   0
1     1  company_category_code     TEXT        0       None   0
2     2   company_country_code     TEXT        0       None   0
3     3     company_state_code     TEXT        0       None   0
4     4         company_region     TEXT        0       None   0
5     5           company_city     TEXT        0       None   0
6     6          investor_name     TEXT        0       None   0
7     7  investor_country_code     TEXT        0       None   0
8     8        investor_region     TEXT        0       None   0
9     9          investor_city     TEXT        0       None   0
10   10     funding_round_type     TEXT        0       None   0
11   11              funded_at     TEXT        0       None   0
12   12           funded_month     TEXT        0       None   0
13   13         funded_quarter     TEXT        0       None   0
14   14            funded_year  INTEGER 

<b>What proportion of the total amount of funds did the top 1% and 10% raise?<b>

In [23]:
conn = sqlite3.connect('invest.db')

q = """with total as (SELECT company_name, SUM(raised_amount_usd) amt 
FROM invest 
WHERE raised_amount_usd is not null
GROUP BY company_name 
ORDER BY 2 DESC)
select SUM(amt) total_amt, COUNT(1) num_customer, 
       COUNT(1) * .10 customer_10_pct,
       COUNT(1) * 0.01 customer_1_pct 
from total 
"""

tot = pd.read_sql(q, conn)
print('-------- SQL solution')
print(tot)

# Calculate TOP 10% 
q = """with total as (SELECT company_name, SUM(raised_amount_usd) amt 
FROM invest
WHERE raised_amount_usd is not null
GROUP BY company_name 
ORDER BY 2 DESC),
top10 As (select company_name, amt from total limit """ 
q += str(int(tot['customer_10_pct'][0])) 
q += """)

SELECT
    CAST((SELECT SUM(amt) from top10) AS FLOAT)/
    (SELECT sum(amt) from total) AS top_10_pct_total_amt_pct
""" 
print()
print(pd.read_sql(q, conn))
print()

# Calculate TOP 1%
q = """with total as (SELECT company_name, SUM(raised_amount_usd) amt 
FROM invest 
WHERE raised_amount_usd is not null
GROUP BY company_name 
ORDER BY 2 DESC),
top1 As (select company_name, amt from total limit """ 
q += str(int(tot['customer_1_pct'][0])) 
q += """)

SELECT
    CAST((SELECT SUM(amt) from top1) AS FLOAT)/
    (SELECT sum(amt) from total) AS top_1_pct_total_amt_pct
""" 

print(pd.read_sql(q, conn))


# Calculate Bottom 10% 
q = """with total as (SELECT company_name, SUM(raised_amount_usd) amt 
FROM invest
WHERE raised_amount_usd is not null
GROUP BY company_name 
ORDER BY 2 ASC),
bottom10 As (select company_name, amt from total limit """ 
q += str(int(tot['customer_10_pct'][0])) 
q += """)

SELECT
    CAST((SELECT SUM(amt) from bottom10) AS FLOAT)/
    (SELECT sum(amt) from total) AS bottom_10_pct_total_amt_pct
""" 
print()
print(pd.read_sql(q, conn))
print()

# Calculate Bottom 1%
q = """with total as (SELECT company_name, SUM(raised_amount_usd) amt 
FROM invest 
WHERE raised_amount_usd is not null
GROUP BY company_name 
ORDER BY 2 DESC),
Bottom1 As (select company_name, amt from total limit """ 
q += str(int(tot['customer_1_pct'][0])) 
q += """)

SELECT
    CAST((SELECT SUM(amt) from Bottom1) AS FLOAT)/
    (SELECT sum(amt) from total) AS bottom_1_pct_total_amt_pct
""" 

print(pd.read_sql(q, conn))




-------- SQL solution
      total_amt  num_customer  customer_10_pct  customer_1_pct
0  681732187973         10357           1035.7          103.57

   top_10_pct_total_amt_pct
0                  0.645395

   top_1_pct_total_amt_pct
0                   0.2504

   bottom_10_pct_total_amt_pct
0                     0.000267

   bottom_1_pct_total_amt_pct
0                      0.2504


<b>Which category of company attracted the most investments?<b>

In [24]:
conn = sqlite3.connect('invest.db')

q = """SELECT company_category_code, SUM(raised_amount_usd) amt 
FROM invest
WHERE raised_amount_usd is not null
GROUP BY company_category_code 
ORDER BY 2 DESC LIMIT 10
"""

print(pd.read_sql(q, conn))

  company_category_code           amt
0               biotech  110396423062
1              software   73084516724
2                mobile   64777379752
3             cleantech   52705225028
4            enterprise   45860927273
5                   web   40143264989
6               medical   25367105281
7           advertising   25076661879
8             ecommerce   22567220071
9       network_hosting   22419683840


<b>Which investor contributed the most money (across all startups)?<b>

In [37]:
conn = sqlite3.connect('invest.db')

q = """SELECT investor_name, SUM(raised_amount_usd) total_amt 
FROM invest
WHERE raised_amount_usd is not null
GROUP BY company_category_code 
ORDER BY 2 DESC LIMIT 1
"""

print(pd.read_sql(q, conn))

   investor_name     total_amt
0  Zenyaku Kogyo  110396423062


<b>Which investors contributed the most money per startup?<b>

In [40]:
conn = sqlite3.connect('invest.db')

q = """
SELECT company_name, investor_name, 
       SUM(raised_amount_usd) total_amt 
FROM invest
WHERE raised_amount_usd is not null
GROUP BY company_category_code, investor_name
ORDER BY company_name, total_amt DESC 
"""

print(pd.read_sql(q, conn))

            company_name                                investor_name  \
0               #waywire                          First Round Capital   
1               #waywire                         Innovation Endeavors   
2               #waywire                                  Jeff Weiner   
3               #waywire                                Oprah Winfrey   
4               #waywire                                  Troy Carter   
5           1000memories                                Caterina Fake   
6                100Plus                               Band of Angels   
7          11i Solutions                  Steel Pier Capital Advisors   
8               121nexus                  National Science Foundation   
9               121nexus                          Mayfly Capital, LLC   
10              121nexus               American Pioneer Ventures, Ltd   
11              121nexus  Providence Economic Development Partnership   
12     1366 Technologies                    U.S. De

<b>Which funding round was the most popular? Which was the least popular?<b>

In [41]:
conn = sqlite3.connect('invest.db')

q = """
with funding AS (
SELECT funding_round_type,  
       SUM(raised_amount_usd) total_amt 
FROM invest
WHERE raised_amount_usd is not null
GROUP BY funding_round_type
),
most_used AS (
    SELECT funding_round_type
    FROM funding
    ORDER BY total_amt DESC LIMIT 1),
least_used AS (
    SELECT funding_round_type
    FROM funding
    ORDER BY total_amt ASC LIMIT 1)
SELECT
    (SELECT funding_round_type FROM most_used) most_used,
    (SELECT funding_round_type FROM least_used) most_used

"""

print(pd.read_sql(q, conn))

   most_used     most_used
0  series-c+  crowdfunding
